## Vector Database setup

Remove old Weaviate DB files.

In [1]:
!rm -rf ~/.local/share/weaviate


### Step 1 - Download sample data

In [2]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data[0])

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [3]:
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

### Step 2 - Create an embedded instance of Weaviate vector database

In [4]:
import weaviate, os
from weaviate import EmbeddedOptions
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-BaseURL": os.environ['OPENAI_API_BASE'],
        "X-OpenAI-Api-Key": openai.api_key  # Replace this with your actual key
    }
)
print(f"Client created? {client.is_ready()}")

Binary /home/jovyan/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.22.3/weaviate-v1.22.3-Linux-amd64.tar.gz
Started /home/jovyan/.cache/weaviate-embedded: process ID 64


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-11-09T10:32:20Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-11-09T10:32:20Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2023-11-09T10:32:20Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2023-11-09T10:32:20Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2023-11-09T10:32:20Z"}


Client created? True


In [5]:
json_print(client.get_meta())

{
  "hostname": "http://127.0.0.1:8079",
  "modules": {
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"
    },
    "text2vec-huggingface": {
      "documentationHref": "https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task",
      "name": "Hugging Face Module"
    },
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "nam

## Step 3 - Create Question collection

In [6]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("Question"):
    client.schema.delete_class("Question")
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # Use OpenAI as the vectorizer
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text",
            "baseURL": os.environ["OPENAI_API_BASE"]
        }
    }
}

client.schema.create_class(class_obj)

{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"question_uENHajf3zQCL","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-11-09T10:33:49Z","took":95191}


## Step 4 - Load sample data and generate vector embeddings

In [7]:
# reminder for the data structure
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [8]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [9]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


## Let's Extract the vector that represents each question!

In [10]:
# write a query to extract the vector for a question
result = (client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(1)
          .do())

json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": [
              -0.003712297,
              0.014701234,
              -0.0030918994,
              -0.009980161,
              0.0026177745,
              0.017741013,
              -0.010181916,
              -0.018938094,
              -0.02143986,
              -0.025596019,
              -0.020027572,
              0.012697131,
              -0.011009113,
              -0.00861495,
              0.016005918,
              0.03402939,
              0.029456269,
              0.008742728,
              0.027035207,
              -0.0036921215,
              -0.01792932,
              0.029429369,
              0.001498033,
              -0.017498907,
              0.004593295,
              0.023524664,
              0.008016409,
              -0.012098591,
              -0.020982549,
              0.027062107,
              -0.008218165,
              0.02645684,
              

## Query time
What is the distance between the `query`: `biology` and the returned objects?

In [11]:
response = (
    client.query
    .get("Question",["question","answer","category"])
    .with_near_text({"concepts": "biology"})
    .with_additional('distance')
    .with_limit(2)
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19695163
          },
          "answer": "DNA",
          "category": "SCIENCE",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.20142835
          },
          "answer": "species",
          "category": "SCIENCE",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        }
      ]
    }
  }
}


In [12]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18959647
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19151002
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.2041738
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.2142275
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
      

## We can let the vector database know to remove results after a threshold distance!

In [13]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"], "distance": 0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.18963969
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19160312
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20419747
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21438634
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

## Vector Databases support for CRUD operations

### Create

In [14]:
#Create an object
object_uuid = client.data_object.create(
    data_object={
        'question':"Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"
    },
    class_name="Question"
 )

In [15]:
print(object_uuid)

db4b74a4-f84c-4c89-891d-d86455645937


### Read

In [16]:
data_object = client.data_object.get_by_id(object_uuid, class_name="Question")
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1699526264070,
  "id": "db4b74a4-f84c-4c89-891d-d86455645937",
  "lastUpdateTimeUnix": 1699526264070,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [17]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
    with_vector=True
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1699526264070,
  "id": "db4b74a4-f84c-4c89-891d-d86455645937",
  "lastUpdateTimeUnix": 1699526264070,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vector": [
    0.022520771,
    -0.013128586,
    -0.0030918075,
    -0.047831047,
    -0.0038867523,
    0.01190018,
    -0.017389618,
    -0.0041234763,
    -0.004564935,
    -0.034651276,
    0.010460642,
    0.025591785,
    -0.0031445906,
    -0.00735124,
    0.010307091,
    -0.0014851233,
    0.013051811,
    0.0054574483,
    0.039488126,
    -0.00806781,
    0.0024696072,
    0.021292364,
    0.0117274355,
    -0.010863713,
    0.0050447807,
    0.0018170167,
    0.016506702,
    -0.032066505,
    0.014932807,
    -0.031836182,
    -0.0078950655,
    0.0007349641,
    -0.00999999,
    -0.013320524,
    -0.018861145,
    -0.014510542,
    -0.0076775355,
    -0.028099779,
    0.008150984,
    0.01

### Update

In [18]:
client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={
        'answer':"Florence, Italy"
    })

In [19]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1699526264070,
  "id": "db4b74a4-f84c-4c89-891d-d86455645937",
  "lastUpdateTimeUnix": 1699526321155,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


### Delete

In [20]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 11
          }
        }
      ]
    }
  }
}


In [21]:
client.data_object.delete(uuid=object_uuid, class_name="Question")

In [22]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}
